In [ ]:
import boto3
import pandas as pd
import glob
import os

In [ ]:
from urllib2 import Request, urlopen
import json
from sqlalchemy import create_engine
import sqlalchemy
query = 'science'
fields = 'headline,body,shortUrl'
api_key  = '6ad864bd-9bb6-4cf1-bdfd-11b5a03c3490'

In [ ]:
def guardian_data(query,fields,api_key):
    '''
    The Guardian API limits developers accounts to 5000 requests.
    If you have a business account feel free to change the request range.
    number_of_pages will have all of the pages for the specified query.
    '''
    url_path = 'https://content.guardianapis.com/search?q={0}&show-fields={1}&api-key={2}'.format(query,fields,api_key)
    request=Request(url_path)
    response = urlopen(request)
    articles = response.read()
    a_json = json.loads(articles)
    number_of_pages = a_json['response']['pages']
    fields_data = []
    for i in range(1,5000):
        if i % 500 == 0:
            print "Loading page {0} out of {1}".format(i,number_of_pages)
        url_path = 'https://content.guardianapis.com/search?q={0}&show-fields={1}&api-key={2}&page={3}'.format(query,fields,api_key,i)
        request = Request(url_path)
        response = urlopen(request)
        articles = response.read()
        a_json = json.loads(articles)
        results = a_json['response']['results']
        for i in range(0,10):
            fields_data.append(dict(results[i])['fields'])
    dataframe = pd.DataFrame(fields_data)
    return dataframe

In [ ]:
query = 'science'
fields = 'headline,body,shortUrl'
api_key  = '6ad864bd-9bb6-4cf1-bdfd-11b5a03c3490'
content_df = guardian_data(query,fields,api_key)

In [ ]:
def clean_html(x):
    soup = BeautifulSoup(x)
    text = soup.get_text()
    return text

In [ ]:
content_df.loc['body'] = content_df['body'].map(lambda x: clean_html(x))

In [ ]:
def to_redshift(username,password,server,db_name,dataframe,table_name):
    print "Starting dataframe upload to database."
    engine = create_engine('postgresql://{0}:{1}@{2}:5439/{3}'.format(username,password,server,db_name))
    dataframe.to_sql(table_name, engine, index = False, chunksize=1000, if_exists = 'replace', dtype=
                             {'body': sqlalchemy.types.NVARCHAR(length=65535), 
                             'headline':  sqlalchemy.types.NVARCHAR(length=255),
                             'shortUrl': sqlalchemy.types.NVARCHAR(length=255)
                             })
    print 'Upload done.'

In [ ]:
username = 'admin'
password = 'password'
server = 'cms-database.cquvdpnoor3i.us-west-2.redshift.amazonaws.com'
db_name = 'dev'
table_name = 'posts'

In [ ]:
to_redshift(username,password,server,db_name,content_df,table_name)